# LABORATORIO #2 - Deteccion de SPAM
 Alfredo Quezada 191002
 
 Randy Venegas   

* #### Exploracion de datos

In [36]:
# agregamos el data set para poder tener una idea de lo que son los datos. 
import pandas as pd 
df1 = pd.read_csv('completeSpamAssassin.csv')
df1.sample(n=5).head()

,Unnamed: 0,Body,Label
1503,1503,(See Disclaimer below)\nDear Internet Marketer...,1
454,454,A:link {TEX-DECORATION: none}A:active {TEXT-DE...,1
3557,3557,[Jeremy Hylton]\n> I think one step towards de...,0
5251,5251,bitbitch@magnesium.net wrote:>I'm also going t...,0
1541,1541,"HELLO,I AM A CITIZEN OF ZAIRE, SON OF THE LATE...",1


In [37]:
# cargamos tambien el segundo conjunto de datos.
df2 = pd.read_csv('enronSpamSubset.csv')
df2.sample(n=5).head()

,Unnamed: 0.1,Unnamed: 0,Body,Label
7601,30074,30074,"Subject: hpl nom for april 21 - 23 , 2001\n ( ...",0
2181,5101,5101,Subject: = ? iso - 8859 - 1 ? q ? win _ notice...,1
8659,20397,20397,Subject: performance review - mid - year\n for...,0
4,1468,1468,"Subject: "" bidstogo "" is places to go , things...",1
5273,20662,20662,Subject: smi 23\n - - - - - - - - - - - - - - ...,0


Ahora que ya sabemos lo que son los dos conjuntos de datasets, procedemos a unificar los dos para crear un data set que contenga ambas caracteristicas. 

In [38]:
#Ahora hacemos el data set final 
#Como se puede ver eliminados los datos que no son relevantes
df1.drop(['Unnamed: 0'], axis=1, inplace=True)
df2.drop(['Unnamed: 0'], axis=1, inplace=True)
df2.drop(['Unnamed: 0.1'], axis=1, inplace=True)

df = pd.concat([df1, df2], ignore_index=True)
df.head()
#Por ultimo guardamos el nuevo data set en un csv final.
df.to_csv('dbfinal.csv', index=False)
db = pd.read_csv('dbfinal.csv')
db.head()


,Body,Label
0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,I thought you might like these:\n1) Slim Down ...,1


Con esta parte realizada, ahora podemos pasar a la parte de preprocesamiento

### Preprocesamiento

In [39]:
# Lo primero es elimanar todos los elementos no deseados y poner los datos en el mismo formato 
db['Body'] = db['Body'].astype(str).str.lower()
db.head()

,Body,Label
0,\nsave up to 70% on life insurance.\nwhy spend...,1
1,1) fight the risk of cancer!\nhttp://www.adcli...,1
2,1) fight the risk of cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,i thought you might like these:\n1) slim down ...,1


In [41]:
#Ahora eliminamos los formatos no deseados 
import re
db['Body']= db['Body'].apply(lambda x: re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", x))
db['Body']= db['Body'].apply(lambda x: re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", x))
db.head()

,Body,Label
0,save up to on life insurance why spend more...,1
1,fight the risk of cancer slim down guar...,1
2,fight the risk of cancer slim down guar...,1
3,...,1
4,i thought you might like these slim down g...,1
